In [1]:
from transformers import (
    AutoConfig, 
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TextStreamer, 
    GenerationConfig, 
    logging,
    TrainingArguments,
    Trainer,
)
import datasets
import json
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import torch
import transformers
import wandb
# from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from pymilvus import Collection, db, connections
import torch.nn as nn
import datasets
import os
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: devasheeshmishra. Use `wandb login --relogin` to force relogin


True

In [3]:
load_dotenv("datasets/synthetic-mt/.env")

True

In [4]:
logging.set_verbosity_info()

In [5]:
_BASE_MODEL_PATH = Path('/mnt/nvme/MODELS/LLM/zephyr-7b-beta/')
_LORA_OUTPUT_PATH = Path('output/loras')

In [6]:
base_model_tokenizer = AutoTokenizer.from_pretrained(_BASE_MODEL_PATH, use_fast=False)
base_model_config = AutoConfig.from_pretrained(_BASE_MODEL_PATH)

loading file tokenizer.model
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading configuration file /mnt/nvme/MODELS/LLM/zephyr-7b-beta/config.json
Model config MistralConfig {
  "_name_or_path": "/mnt/nvme/MODELS/LLM/zephyr-7b-beta",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.37.0.dev0",
  "use_cache": true,
  "vocab_size": 32000
}



In [7]:
base_model = AutoModelForCausalLM.from_pretrained(
    _BASE_MODEL_PATH,
    config=base_model_config, 
    device_map='cuda:1', 
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

loading weights file /mnt/nvme/MODELS/LLM/zephyr-7b-beta/pytorch_model.bin.index.json
Instantiating MistralForCausalLM model under default dtype torch.float16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 2
}



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at /mnt/nvme/MODELS/LLM/zephyr-7b-beta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file /mnt/nvme/MODELS/LLM/zephyr-7b-beta/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [8]:
for param in base_model.parameters():
    # Turning off gradient calculation for base model as we want to train lora, not base model
    param.requires_grad = False

In [9]:
base_model.config.use_cache = False

In [10]:
base_model_tokenizer.bos_token, base_model_tokenizer.pad_token, base_model_tokenizer.eos_token, base_model_tokenizer.unk_token

('<s>', '</s>', '</s>', '<unk>')

## HYPER PARAMETERS

In [11]:
SYSTEM_PROMPT = "You are a Personal Assistant named JARVIS. You are installed in Devasheesh Mishra's House and running on his servers. You are integrated in his smart home system. If he is generally talking about something, just talk to him but if he is asking to control the state of a device, you first need to ask the system to give you the functions needed to control the device, then you can do the appropriate function call to control devices. You can also ask user for more information if needed. You can also use tools like 1. calculator (for doing basic mathematical calculations i.e. addition, substraction, multiplication, division) 2. calendar (to find todays date) 3. clock (to find todays day and current time). You can also use Google to search for Information online. Give short responses as much as possible. Be attentive to user commands and inquiries, ensuring a seamless and efficient smart home experience. You are designed to make his life easier and better."

In [12]:
BATCH_SIZE = 1
EPOCHS = 2
WARMUP_STEPS = 100
TOTAL_STEPS = 1000
LEARNING_RATE = 4e-5

## Setup Dataset

Base Zephyr Model Prompt Template:
```text
<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!
```

In [13]:
class MilvusDataset(Dataset):
    def __init__(self):
        connections.connect(
            alias="default",
            host = os.getenv("MILVUS_HOST"),
            port = 19530,
            user=os.getenv("MILVUS_USER"),
            password=os.getenv("MILVUS_PASSWORD"),
        )
        db.using_database("JARVIS")
        self.collection = Collection("calculator")
        self.data_itrator = self.collection.query_iterator(
                                batch_size=1,
                                output_fields=["conversation"],
                                expr="conversation_id > 0",
                            )

    def __len__(self):
        return self.collection.num_entities

    def __getitem__(self, idx):
        conversation: dict = self.data_itrator.next()[0]
        conversation = self.apply_prompt_template(conversation)
        return conversation
        # return {
        #         'input_ids': torch.tensor(conversation['input_ids'], device=base_model.device), 
        #         'attention_mask': torch.tensor(conversation['attention_mask'], device=base_model.device)
        #     }

    def apply_prompt_template(self, conversation: dict):
        # add </s> at end of lines which do not end with </s>
        conversation["conversation"]["data"] = ''.join([x + '</s>\n' if not x.strip().endswith("</s>") else x + '\n' for x in conversation["conversation"]["data"].split("\n")])

        # apply zephyr-7b-beta chat template
        prompt = f"<|system|>\n{SYSTEM_PROMPT}</s>\n{conversation["conversation"]["data"].replace("USER: ", "<|user|>\n").replace("ASSISTANT: ", "<|assistant|>\n")}"
        # return prompt
        return base_model_tokenizer(prompt, return_tensors="pt", padding=True).to(base_model.device)

## Train Model

### [Gradient Accumulation](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-accumulation)
    The idea behind gradient accumulation is to instead of calculating the gradients for the whole batch at once to do it in smaller steps. The way we do that is to calculate the gradients iteratively in smaller batches by doing a forward and backward pass through the model and accumulating the gradients in the process. When enough gradients are accumulated we run the model’s optimization step. This way we can easily increase the overall batch size to numbers that would never fit into the GPU’s memory. In turn, however, the added forward and backward passes can slow down the training a bit.

### [Gradient Checkpointing](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing)
    Even when we set the batch size to 1 and use gradient accumulation we can still run out of memory when working with large models. In order to compute the gradients during the backward pass all activations from the forward pass are normally saved. This can create a big memory overhead. Alternatively, one could forget all activations during the forward pass and recompute them on demand during the backward pass. This would however add a significant computational overhead and slow down training.

    Gradient checkpointing strikes a compromise between the two approaches and saves strategically selected activations throughout the computational graph so only a fraction of the activations need to be re-computed for the gradients. See this great article explaining the ideas behind gradient checkpointing.

In [14]:
# base_model.gradient_checkpointing_enable()
base_model.enable_input_require_grads()

In [15]:
# def print_trainable_parameters(model: nn.Module):
#     """
#     Prints the number of trainable parameters in the model.
#     """
#     trainable_params = 0
#     all_param = 0
#     for name, param in model.named_parameters():
#         all_param += param.numel()
#         if param.requires_grad:
#             trainable_params += param.numel()
#     print(
#         f"trainable params: {trainable_params} || all params: {all_param} || trainable(%): {100 * trainable_params / all_param}"
#     )

https://medium.com/@manyi.yim/more-about-loraconfig-from-peft-581cf54643db

In [16]:
lora_config = LoraConfig(
    # peft_type: str | PeftType = None,
    # auto_mapping: dict | None = None,
    # base_model_name_or_path: str = None,
    # revision: str = None,
    task_type = TaskType.CAUSAL_LM,
    # inference_mode: bool = False,
    r = 16, #! 8, 16, 32, 64
    target_modules = ["q_proj", "v_proj"],
    lora_alpha = 32, #! 8, 16, 32
    lora_dropout = 0.05, #! 0.05
    # fan_in_fan_out: bool = False,
    bias = "none",
    # modules_to_save: List[str] | None = None,
    # init_lora_weights: bool = True,
    # layers_to_transform: List[int] | int | None = None,
    # layers_pattern: str | None = None
)
peft_model = get_peft_model(base_model, lora_config, adapter_name='jarvis-calculator-v0_1')
peft_model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940290959023318


In [17]:
trainig_parms = TrainingArguments(
    # gradient_checkpointing=True,
    # gradient_checkpointing_kwargs={"use_reentrant":False},
    gradient_accumulation_steps=1,
    per_device_train_batch_size=BATCH_SIZE,
    
    warmup_steps=WARMUP_STEPS,
    max_steps=TOTAL_STEPS,

    # optim="adamw_torch",
    learning_rate=LEARNING_RATE,

    fp16=True,
    logging_steps=1,
    output_dir=_LORA_OUTPUT_PATH,
    
    report_to="wandb",

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    eval_steps=10,
    evaluation_strategy="steps",
    greater_is_better=False,
    
    save_steps=10,
    
    save_total_limit=2,

    # save_only_model=True,
)

PyTorch: setting up devices


In [18]:
data = MilvusDataset()

In [19]:
dataset = [next(iter(data)) for _ in range(len(data))]

In [20]:
train_dataset = dataset[:int(len(dataset) * 0.8)]
eval_dataset = dataset[int(len(dataset) * 0.8):]

In [21]:
early_stop = transformers.EarlyStoppingCallback(10, 1.15)

In [22]:
trainer = Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=trainig_parms,
    tokenizer=base_model_tokenizer,
    callbacks=[early_stop],
    data_collator=transformers.DataCollatorForLanguageModeling(base_model_tokenizer, mlm=False),
    # data_collator=lambda data: {
    #     'input_ids': torch.stack([item['input_ids'] for item in data]),
    #     'attention_mask': torch.stack([item['attention_mask'] for item in data])
    # },
)

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [23]:
dataset[0]['input_ids'].device

device(type='cuda', index=1)

In [25]:
trainer.train()

In [ ]:
from datasets import load_dataset
ds = load_dataset('birgermoell/open_assistant_dataset')

In [ ]:
base_model_tokenizer.add_eos_token = True
base_model_tokenizer.padding_side = "left"

In [ ]:
# Define cleaning and preprocessing functions.
def text_to_dialogue(text):
    return [sentence.replace('User:', '').replace('Chip:', '').strip() for sentence in text.split('Assistant:')]

def dialogue_to_chat(dialogue):
    out = [{'role': 'system', 'content': 'You are a friendly chatbot assistant.'}]
    for idx, message in enumerate(dialogue):
        role = 'user' if idx%2==0 else 'assistant'
        out.append({'role': role, 'content': message})
    return out

def chat_to_input(chat):
    return base_model_tokenizer.apply_chat_template(chat, tokenize=False)

def process_example(example):
    out = text_to_dialogue(example)             # Clean up data from redundant words
    out = dialogue_to_chat(out)                 # Convert to ChatML format
    out = chat_to_input(out)                    # Add a task description & Apply base model chat template
    return out

In [ ]:
import pandas as pd
df = ds['train'].to_pandas()

In [ ]:
# Apply on dataset.
data = list(map(process_example, df['text']))

# Shuffle dataset.
from random import shuffle
shuffle(data)

# Tokenize data.
tokenized_data = list(map(base_model_tokenizer, data))

In [ ]:
split_idx = int(.99 * len(data))
train_data, val_data = tokenized_data[:split_idx], tokenized_data[split_idx:]

In [ ]:
print(base_model_tokenizer.decode(dataset[0]['input_ids'])), print(base_model_tokenizer.decode(train_data[811]['input_ids']))